In [5]:
import os
import pandas as pd
import numpy as np
from scipy import spatial

# Reading data

In [6]:
rating_data = pd.read_csv("", encoding='utf-8', delimiter=";")
books_data = pd.read_csv("", delimiter=";", encoding='utf-8', error_bad_lines=False)
users = pd.read_csv("", delimiter=";")

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 25129

# Preprocess

In [38]:
# renaming 
books_data.rename(columns={'ISBN':'isbn',
                           'Book-Title':'title',
                           'Book-Author':'author',
                           'Year-Of-Publication':'year',
                           'Publisher':'publisher',
                           'Image-URL-S':'image_s',
                           'Image-URL-M':'image_m',
                           'Image-URL-L':'image_l'},
                  inplace=True) 
rating_data.rename(columns={'User-ID':'uid', 
                            'ISBN':'isbn', 
                            'Book-Rating':'rating'},
                  inplace=True)

users.rename(columns={'User-ID':'uid', 
                      'Location':'location', 
                      'Age':'age'},
            inplace=True)
# selecting only users who looked at more than 150 books
top_users = rating_data['uid'].value_counts() >= 150
top_users_index = top_users[top_users].index

# selecting all the user ratings who looked at more than 150 books
selected_ratings = rating_data[rating_data['uid'].isin(top_users_index)]
final_df = selected_ratings.merge(books_data, on = "isbn", how = "inner", sort=False)

In [8]:
# counting number ratings each book has
rating_count = final_df.groupby('title')['rating'].count().reset_index() 

# merging datasets to have counts
final_df = final_df.merge(rating_count, on='title')
final_df.rename(columns={'rating_y':'rating_count',
                        'rating_x':'rating'}, inplace = True)

# select only books which have count of more than 50
final_df = final_df[final_df['rating_count'] >= 50]

# dropping duplicates
final_df.drop_duplicates(['uid','title'],inplace=True)
final_df.shape

(76070, 11)

# Pivot transformation

In [11]:
book_matrix = final_df.pivot_table(columns='uid', index='title', values='rating')
book_matrix.fillna(0,inplace=True)
book_matrix.head()

uid,254,1733,1903,2033,2110,2276,2766,2891,2977,3363,...,274308,274808,275970,276018,276680,277427,277478,277639,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model building

In [42]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_matrix)

In [43]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute', n_neighbors=5) ## model
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
# selecting random book 
n = np.random.choice(book_matrix.shape[0])
print("n:", n)
pd.DataFrame(book_matrix.iloc[n, :]).columns

In [ ]:
#The Two Towers (The Lord of the Rings, Part 2)
book_matrix[book_matrix.index == "Harry Potter and the Sorcerer's Stone (Book 1)"].values

In [ ]:
# this comes from the flask app 
user_titles = ["Harry Potter and the Sorcerer's Stone (Book 1)", 'The Notebook', 'The Secret Garden']

# Recommendations

In [ ]:
# distances, suggestions = model.kneighbors(book_pivot[book_pivot.index == "The Two Towers (The Lord of the Rings, Part 2)"].values.reshape(1, -1))
# "The Secret Garden"
def recommend(book_pivot, title):
    
    suggestions = model.kneighbors(book_pivot[book_pivot.index == title]\
                                              .values.reshape(1, -1), return_distance=False)
    suggestions = suggestions[0][1:]
    rec_list = book_pivot.index[suggestions]
    return rec_list.tolist()

recommended_list = [recommend(book_matrix, a_title) for a_title in user_titles]
recommended_list = list(np.concatenate(recommended_list))


In [ ]:
# top_10 = ['The Notebook', 'Circle of Friends', 'Rebecca','The Secret Garden',
#           'The Two Towers (The Lord of the Rings, Part 2)', 'Interview with the Vampire',
#           'Jurassic Park', 'The Secret Life of Bees', "Harry Potter and the Sorcerer's Stone (Book 1)",
#           'Scarlet Feather']

url_list = []
for book_title in recommended_list:
    book_image = final_df[final_df['title'] == book_title][['image_l','author', 'year']].reset_index(drop=True)
    url_list.append(book_image.iloc[0].to_dict())
